In [1]:
import numpy as np
import uproot
import h5py
import os
import torch
import torch_geometric.nn as PyG
from torch_geometric.transforms import Distance
from torch_geometric.data import InMemoryDataset as PyGDataset, Data as PyGData
from torch_geometric.data import Data
import math
import numba
import numpy as np
import torch
import h5py
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as PyG
from torch_geometric.transforms import Distance
from torch_geometric.data import DataLoader
from torch_geometric.data import Data as PyGData
from torch_geometric.data import Data
import sys, os
import subprocess
import csv, yaml
import math
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import torch.optim as optim
import argparse
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.tri as tri
# import numpy, numba, awkward, awkward.numba

In [2]:

# path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex2/220323_084833/delphes'
# graph_path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex2/220323_084833/graph/'
# output_path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex2/220323_084833/h5/'


# path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex1/220324_055935/delphes'
# graph_path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex1/220324_055935/graph/'
# output_path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex1/220324_055935/h5_2/'


# path = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2/'
path = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520/'
# path = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_0/'

# out_path1 = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_w1/'
out_path1 = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/'
# out_path1 = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_0_w1/'

# out_path2 = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_w2/'
out_path2 = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w2/'
# out_path2 = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_0_w2/'

In [3]:
resampling_model_9s = '/users/yewzzang/work/HEPGNN_4top_resampling/result/date20220629_resampling_fea4_9s_test1'
resampling_model_all = '/users/yewzzang/work/HEPGNN_4top_resampling/result/date20220629_resampling_fea4_all_test1'

In [4]:
sys.path.append("./python")

In [5]:
from model.allModel import *

model_9s = torch.load(resampling_model_9s+'/model.pth', map_location='cpu')
model_9s.load_state_dict(torch.load(resampling_model_9s+'/weight.pth', map_location='cpu'))
model_9s = model_9s.cuda()
device = 'cuda'

model_all = torch.load(resampling_model_all+'/model.pth', map_location='cpu')
model_all.load_state_dict(torch.load(resampling_model_all+'/weight.pth', map_location='cpu'))
model_all = model_all.cuda()
device = 'cuda'

In [6]:
res = []
for root, dirs, files in os.walk(path):
    rootpath = os.path.join(os.path.abspath(path), root)
    for file in files:
        filepath = os.path.join(rootpath, file)
        
        filetype = filepath.split('.')[-1]
        if filetype != 'pt': continue
  
        res.append(filepath)

In [7]:
res

['/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520/output_ex1_v2-powheg-61.pt',
 '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520/output_ex1_v2-powheg-58.pt',
 '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520/output_ex1_v2-powheg-43.pt',
 '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520/output_ex1_v2-powheg-193.pt',
 '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520/output_ex1_v2-powheg-156.pt',
 '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520/output_ex1_v2-powheg-32.pt',
 '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520/output_ex1_v2-powheg-196.pt',
 '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520/output_ex1_v2-powheg-62.pt',
 '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520/output_ex1_v2-powheg-161.pt',
 '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520/output_ex1_v2-powheg-24.pt',
 '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v

In [8]:
import time
start = time.time()
for i in range(len(res)):

    file_name = res[i]
    cla = file_name.split('/')[-4].split('_')
    num = file_name.split('/')[-1].split('.')[-2]

    

    save_f_name = num

    save_path1 = out_path1
    save_path2 = out_path2
#     if not os.path.exists(save_path): os.makedirs(save_path)
    save_file_name1 = save_path1 + save_f_name + '.pt'
    save_file_name2 = save_path2 + save_f_name + '.pt'
#     print(save_file_name)
### load data
    try:
        r_data = torch.load(file_name)
#     r_data = uproot.open(file_name)["Delphes"]
    except KeyError:
        print("delphes 없음")
        pass
    

    model_9s.eval()
    model_all.eval()
    
    
    list_9s = []
    list_all = []
    for i in range(len(r_data)):
#     for i in range(1):
        data = r_data[i].to(device)
        data_1 = PyGData(x = data.x[:,:4], edge_index=data.edge_index, pos = data.pos, y = data.y, batch = torch.from_numpy(np.zeros([1],dtype = np.int)))
        data_1 = data_1.to(device)
        
        pred1 = model_9s(data_1)
        pred2 = model_all(data_1)
        
        data_9s = data
        data_all = data
        

        data_9s.rs = pred1[0].detach().to("cpu")
        data_9s.x = data.x.detach().to("cpu")
        data_9s.edge_index = data.edge_index.detach().to("cpu")
        data_9s.pos = data.pos.detach().to("cpu")
        data_9s.y = data.y.detach().to("cpu")
        
        data_all.rs = pred2[0].detach().to("cpu")
        data_all.x = data.x.detach().to("cpu")
        data_all.edge_index = data.edge_index.detach().to("cpu")
        data_all.pos = data.pos.detach().to("cpu")
        data_all.y = data.y.detach().to("cpu")
        
        list_9s.append(data_9s)
        list_all.append(data_all)
        
    torch.save(list_9s, save_file_name1)
    torch.save(list_all, save_file_name2)
    
    print(save_file_name1,'finish')
    print(save_file_name1,'finish')
    
print(time.time()-start)

/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-61.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-61.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-58.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-58.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-43.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-43.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-193.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-193.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-156.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-156.pt fin

/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-180.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-180.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-136.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-136.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-158.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-158.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-183.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-183.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-121.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-121.

/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-106.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-106.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-9.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-9.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-178.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-178.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-94.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-94.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-177.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-177.pt fin

/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-174.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-174.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-4.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-4.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-150.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-150.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-36.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-36.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-81.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-81.pt finis

/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-112.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-112.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-134.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-134.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-167.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-167.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-108.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-108.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-184.pt finish
/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/output_ex1_v2-powheg-184.